In [ ]:
!git clone https://github.com/thewh1teagle/optispeech -b he-v1
%cd optispeech

In [ ]:
%env RYE_INSTALL_OPTION=--yes
!curl -sSf https://rye.astral.sh/get | bash

!bash -c "$(wget -O - https://apt.llvm.org/llvm.sh)"
!sudo apt install cmake nvtop htop p7zip-full aria2 -y
!sudo ln -s /usr/bin/clang-18 /usr/bin/clang
!sudo ln -s /usr/bin/clang++-18 /usr/bin/clang++

In [ ]:
!/root/.rye/shims/rye sync
!/root/.rye/shims/rye add torchaudio==2.3.1

In [ ]:
# !aria2c -x16 -s16 https://github.com/Aunali321/optispeech-train/releases/download/v0.1.0/last.ckpt.7z
# !aria2c -x16 -s16 https://github.com/Aunali321/optispeech-train/releases/download/v0.1.0/guj_dataset.7z

!gh release download v0.1.0 -R https://github.com/Aunali321/optispeech-train --pattern "last.ckpt.7z"
!gh release download v0.1.0 -R https://github.com/Aunali321/optispeech-train --pattern "guj_dataset.7z"
!gh release download v0.1.0 -R https://github.com/Aunali321/optispeech-train --pattern "guj.yaml"
!gh release download v0.1.0 -R https://github.com/Aunali321/optispeech-train --pattern "gujju.yaml"

!7z x last.ckpt.7z
!7z x guj_dataset.7z

!mkdir data
!mv guj_dataset data
!mv guj.yaml configs/data/guj.yaml
!mv gujju.yaml configs/experiment/gujju.yaml

In [ ]:
!pip install huggingface_hub[cli]
!huggingface-cli login --token your_token

In [8]:
!sed -i "s|/root/israwave/lab/optispeech/|$(pwd)/|g" ./data/guj_dataset/val.txt ./data/guj_dataset/train.txt

In [ ]:
# Single GPU First Run
!.venv/bin/python -m optispeech.train \
    experiment="gujju" \
    model.train_args.evaluate_utmos=false \
    data.batch_size=12 \
    data.num_workers=12 \
    data.train_filelist_path="data/guj_dataset/train.txt" \
    data.valid_filelist_path="data/guj_dataset/val.txt" \
    paths.log_dir="data/guj_dataset/logs" \
    callbacks.model_checkpoint.every_n_epochs=5 \
    callbacks.model_checkpoint.save_last=True \
    callbacks.model_checkpoint.save_on_train_epoch_end=True \
    trainer.check_val_every_n_epoch=25

In [ ]:
# # Single GPU Resume training
!.venv/bin/python -m optispeech.train \
    experiment="gujju" \
    model.train_args.evaluate_utmos=false \
    data.batch_size=12 \
    data.num_workers=12 \
    data.train_filelist_path="data/guj_dataset/train.txt" \
    data.valid_filelist_path="data/guj_dataset/val.txt" \
    paths.log_dir="data/guj_dataset/logs" \
    callbacks.model_checkpoint.every_n_epochs=5 \
    callbacks.model_checkpoint.save_last=True \
    callbacks.model_checkpoint.save_on_train_epoch_end=True \
    trainer.check_val_every_n_epoch=25 \
    ckpt_path="last.ckpt"

In [ ]:
# Multi GPU First Run
!.venv/bin/python -m optispeech.train \
    experiment="gujju" \
    model.train_args.evaluate_utmos=false \
    data.batch_size=12 \
    data.num_workers=48 \
    data.train_filelist_path="data/guj_dataset/train.txt" \
    data.valid_filelist_path="data/guj_dataset/val.txt" \
    paths.log_dir="data/guj_dataset/logs" \
    callbacks.model_checkpoint.every_n_epochs=5 \
    callbacks.model_checkpoint.save_last=True \
    callbacks.model_checkpoint.save_on_train_epoch_end=True \
    trainer=ddp \
    trainer.devices=[0] \
    trainer.strategy=ddp_find_unused_parameters_true \
    trainer.accelerator=gpu \
    trainer.sync_batchnorm=True \
    trainer.check_val_every_n_epoch=25

In [ ]:
# Multi GPU Resume training
!.venv/bin/python -m optispeech.train \
    experiment="gujju" \
    model.train_args.evaluate_utmos=false \
    data.batch_size=12 \
    data.num_workers=48 \
    data.train_filelist_path="data/guj_dataset/train.txt" \
    data.valid_filelist_path="data/guj_dataset/val.txt" \
    paths.log_dir="data/guj_dataset/logs" \
    callbacks.model_checkpoint.every_n_epochs=5 \
    callbacks.model_checkpoint.save_last=True \
    callbacks.model_checkpoint.save_on_train_epoch_end=True \
    trainer=ddp \
    trainer.devices=[0] \
    trainer.strategy=ddp_find_unused_parameters_true \
    trainer.accelerator=gpu \
    trainer.sync_batchnorm=True \
    trainer.check_val_every_n_epoch=25 \
    ckpt_path="last.ckpt"

In [ ]:
#!7z a last.ckpt.7z last.ckpt
#!gh release upload v0.1.0 last.ckpt.7z -R https://github.com/Aunali321/optispeech-train --clobber

!7z a last.ckpt.7z last.ckpt && gh release upload v0.1.0 last.ckpt.7z -R https://github.com/Aunali321/optispeech-train --clobber && rm last.ckpt.7z
!7z a 'checkpoint_epoch=819_step=337440.ckpt.7z' 'checkpoint_epoch=819_step=337440.ckpt' && gh release upload v0.1.0 checkpoint_epoch=819_step=337440.ckpt.7z -R https://github.com/Aunali321/optispeech-train --clobber && rm checkpoint_epoch=819_step=337440.ckpt.7z